# AoC 2024 - Day 9

<https://adventofcode.com/2024/day/9>

In [1]:
from icecream import ic
import time

In [2]:
# use_test = True  # Comment out for using actual puzzle input

## Part 1

In [3]:
test_data1 = """12345
"""

test_data2 = """2333133121414131402
"""

try:
    use_test
except NameError:
    use_test = False
    ic("use_test was undefined - forcing to", use_test)
    
if use_test:
    assert test_data1 != None
    assert test_data2 != None

ic| 'use_test was undefined - forcing to', use_test: False


In [4]:
# Read the puzzle input into a list of strings, one per line
#
if use_test:
    input_lines = test_data2.splitlines()
else:
    with open("input_day09_gmacario.txt", 'r') as file:
        input_lines = [line.rstrip() for line in file]

ic(time.ctime())
if use_test:
    ic(input_lines)

ic| time.ctime(): 'Sat Dec 14 13:09:10 2024'


In [5]:
def convert_input_to_string(input_str:str, verbose=False) -> str:
    # ic(input_str)
    result = ""
    is_free_space = False
    file_id = 0
    for ch in input_str.rstrip():
        sz = int(ch)
        if is_free_space:
            if verbose:
                print(f"DEBUG: {sz} block(s) of free space")
            result += "." * sz
        else:
            if verbose:
                print(f"DEBUG: {sz} block(s) file (file_id {file_id})")
            # result += f"({str(file_id)})" * sz
            result += str(file_id) * sz
            file_id += 1
        is_free_space = not is_free_space
    return result[:]

In [6]:
def convert_input_to_map(input_str:str, verbose=False) -> list:
    # ic(input_str)
    result = list()
    is_free_space = False
    file_id = 0
    for ch in input_str.rstrip():
        sz = int(ch)
        if is_free_space:
            if verbose:
                print(f"DEBUG: {sz} block(s) of free space")
            result.extend(["."] * sz)
        else:
            if verbose:
                print(f"DEBUG: {sz} block(s) file (file_id {file_id})")
            result.extend([file_id] * sz)
            file_id += 1
        is_free_space = not is_free_space
    return result[:]

In [7]:
# convert_input_to_string(test_data1, verbose=True)

In [8]:
# x = convert_input_to_map(test_data2, verbose=True)
# print(x)

In [9]:
# convert_input_to_map(test_data2, verbose=False)

In [10]:
# disk_map = convert_input_to_map(test_data2)
# print(disk_map)

In [11]:
def compute_checksum(disk_map: list) -> int:
    result = 0
    position = 0
    for block in disk_map:
        if block != ".":
            result += block * position
        position += 1
    return result

In [12]:
def sort_disk_map(disk_map: list) -> list:
    # ic(len(disk_map))
    for pos in range(len(disk_map)-1,-1,-1):
        if disk_map[pos] != ".":
            # ic(pos)
            for pos2 in range(pos):
                if disk_map[pos2] == ".":
                    disk_map[pos], disk_map[pos2] = disk_map[pos2], disk_map[pos]
                else:
                    pass
        else:
            pass
    
    # print(disk_map)
    return disk_map

In [13]:
# input_lines

In [14]:
disk_map = convert_input_to_map(input_lines[0],verbose=False)
# print(disk_map)

In [15]:
c0 = compute_checksum(disk_map)

In [16]:
def solve_part1(disk_map):
    sort_disk_map(disk_map)
    result = compute_checksum(disk_map)
    return result

tm_begin = time.time()
part1_result = solve_part1(disk_map)
tm_end = time.time()
ic("elapsed=", tm_end - tm_begin, "result=", part1_result)
print("Day 09 Part 1 RESULT:")
print(part1_result)

ic| "elapsed=": 'elapsed='
    tm_end - tm_begin: 132.87688851356506
    "result=": 'result='
    part1_result: 6360094256423


Day 09 Part 1 RESULT:
6360094256423


## Part 2

In [17]:
def len_free_block_at(start:int) -> int:
    """
    Return the length of a free block starting at start
    """
    assert 0 <= start < len(disk_map)
    for pos in range(start,len(disk_map)):
        if disk_map[pos] != ".":
            return pos - start
    return 0

# ic(len_free_block_at(2))
# ic(len_free_block_at(7))
# ic(len_free_block_at(8))
# print(len(disk_map),disk_map)
# ic(len_free_block_at(31))

In [18]:
def first_free_cluster_with_at_least(num_blocks:int) -> tuple:
    """
    Find the first free space with at least num_blocks_fileno
    returns a tuple (pos, l)
    """
    for pos in range(0, len(disk_map)-num_blocks):
        # ic("Trying at", pos)
        l = len_free_block_at(pos)
        # ic("Got", l)
        if l >= num_blocks:
            return (pos, l)
    return (-1, 0)

# ic(first_free_cluster_with_at_least(3))
# ic(first_free_cluster_with_at_least(4))

In [19]:
# Restore disk_map after Part 1
disk_map = convert_input_to_map(input_lines[0],verbose=False)

# print(disk_map)

# Find the largest file_id number
largest_fileno = max(((id if id != "." else -1) for id in disk_map))
ic(largest_fileno)

for fileno in range(largest_fileno,0,-1):
    if fileno % 100 == 0:
        ic("Handling", fileno)

    first_block_fileno = disk_map.index(fileno)
    # Find how many blocks are required by fileno
    num_blocks_fileno = disk_map.count(fileno)
    # ic(fileno, first_block_fileno, num_blocks_fileno)
    # ic("step 010", disk_map)
    first_empty_pos = first_free_cluster_with_at_least(num_blocks_fileno)[0]
    # ic("step 020", first_empty_pos)

    if first_empty_pos < 0:
        # ic("Cannot move", fileno)
        continue

    if first_empty_pos > first_block_fileno:
        # ic("Empty block is after file")
        continue

    # ic("Zeroing old file blocks of disk_map")
    disk_map = [blk if blk != fileno else "." for blk in disk_map]

    # ic("Saving file blocks in empty space")
    for k in range(first_empty_pos, first_empty_pos + num_blocks_fileno):
        disk_map[k] = fileno

# _ = ic("After sorting2", disk_map)

c2 = compute_checksum(disk_map)
ic(c2)

ic| largest_fileno: 9999
ic| 'Handling', fileno: 9900
ic| 'Handling', fileno: 9800
ic| 'Handling', fileno: 9700
ic| 'Handling', fileno: 9600
ic| 'Handling', fileno: 9500
ic| 'Handling', fileno: 9400
ic| 'Handling', fileno: 9300
ic| 'Handling', fileno: 9200
ic| 'Handling', fileno: 9100
ic| 'Handling', fileno: 9000
ic| 'Handling', fileno: 8900
ic| 'Handling', fileno: 8800
ic| 'Handling', fileno: 8700
ic| 'Handling', fileno: 8600
ic| 'Handling', fileno: 8500
ic| 'Handling', fileno: 8400
ic| 'Handling', fileno: 8300
ic| 'Handling', fileno: 8200
ic| 'Handling', fileno: 8100
ic| 'Handling', fileno: 8000
ic| 'Handling', fileno: 7900
ic| 'Handling', fileno: 7800
ic| 'Handling', fileno: 7700
ic| 'Handling', fileno: 7600
ic| 'Handling', fileno: 7500
ic| 'Handling', fileno: 7400
ic| 'Handling', fileno: 7300
ic| 'Handling', fileno: 7200
ic| 'Handling', fileno: 7100
ic| 'Handling', fileno: 7000
ic| 'Handling', fileno: 6900
ic| 'Handling', fileno: 6800
ic| 'Handling', fileno: 6700
ic| 'Handling', fi

6379677752410

In [20]:
def solve_part2():
    return c2

# tm_begin = time.time()
part2_result = solve_part2()
tm_end = time.time()
ic("elapsed=", tm_end - tm_begin, "result=", part2_result)
print("Day 09 Part 2 RESULT:")
print(part2_result)

ic| "elapsed=": 'elapsed='
    tm_end - tm_begin: 322.2749342918396
    "result=": 'result='
    part2_result: 6379677752410


Day 09 Part 2 RESULT:
6379677752410
